In [1]:
from textblob import TextBlob
import pandas as pd
import pymongo
from nltk.stem import WordNetLemmatizer
import numpy as np  
import re  

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define the 'classDB' database in Mongo
db = client.projDB
tweet = db.char_hashtags.find()

In [3]:
df = pd.DataFrame(columns=["character_lower", "character", "text", "sentiment", "polarity", "subjectivity", "Preprocess"])
df1 = df

In [4]:
count = 0
for each_tweet in tweet:
    analysis = TextBlob(each_tweet["text"])
    senti = ""
    polarity = 0.0
    subjectivity = 0.0
    
    if analysis.sentiment[0]>0:
       senti = "Pos"
    elif analysis.sentiment[0]<0:
       senti = "Neg"
    else:
       senti = "Neu"
    
    polarity = float(analysis.sentiment[0])
    subjectivity = float(analysis.sentiment[1])
    
    if each_tweet["entities"]["hashtags"] is None:
        char_name = ""
    else:
        if len(each_tweet["entities"]["hashtags"]) > 0:
            char_name = each_tweet["entities"]["hashtags"][0]["text"]
    
    try: 
        df1 = pd.DataFrame({"character_lower": char_name.lower(), "character": char_name, "text": each_tweet["text"], "sentiment": [senti], "polarity": [polarity], "subjectivity": [subjectivity], "Preprocess": ""})
        df = df.append(df1, ignore_index=True)
    except:
        pass
    count += 1
    if count > 100:
        break

In [5]:
stemmer = WordNetLemmatizer()

for index, row in df.iterrows():
    # Remove all the special characters
    test_preproc = re.sub(r'\W', ' ', str(row["text"]))

    # remove all single characters
    test_preproc = re.sub(r'\s+[a-zA-Z]\s+', ' ', test_preproc)

    # Remove single characters from the start
    test_preproc = re.sub(r'\^[a-zA-Z]\s+', ' ', test_preproc) 

    # Substituting multiple spaces with single space
    test_preproc = re.sub(r'\s+', ' ', test_preproc, flags=re.I)

    # Removing prefixed 'b'
    test_preproc = re.sub(r'^b\s+', '', test_preproc)

    # Converting to Lowercase
    test_preproc = test_preproc.lower()
    
    # Lemmatization
    test_preproc = test_preproc.split()

    test_preproc = [stemmer.lemmatize(word) for word in test_preproc]
    comb_string = ' '.join(test_preproc)
    df.loc[index, "Preprocess"] = comb_string

In [6]:
df.head()

,character_lower,character,text,sentiment,polarity,subjectivity,Preprocess
0,gameofthrones,gameofthrones,RT @kbrxci: Look at their faces...They've warn...,Neu,0.0,0.0,rt kbrxci look at their face they ve warned u ...
1,gameofthrones,GameofThrones,RT @isthatdejavuu: Sansa in Winterfell watchin...,Neg,-0.2,0.8,rt isthatdejavuu sansa in winterfell watching ...
2,gameofthrones,GameofThrones,RT @moashrav: Spoilers without context\n#Gameo...,Neu,0.0,0.0,rt moashrav spoiler without context gameofthro...
3,gameofthrones,GameofThrones,RT @CriticalBard: So... you’re telling me... s...,Neu,0.0,0.0,rt criticalbard so you re telling me she could...
4,madqueen,MadQueen,RT @HbWINGer27: A close up of Varys' letter ab...,Neu,0.0,0.0,rt hbwinger27 close up of varys letter about d...


In [8]:
df.to_csv("data/test.csv", index=False)